To je ista koda kot za 2del_subpath_SA samo da je narejena za geodesic number.
Zapisuje v 2del_geodesic_SA.csv

Zacetne podatke sva vzela iz ...

In [ ]:
import random as pyrandom
from sage.all import *
import pandas as pd
import os

# ============================================================
#  GEODESIC NUMBER (OPTIMIZIRANO)
# ============================================================

def geodesic_subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    dist = G.distance_all_pairs()
    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited, length):
            u_idx = idx[u]

            if length == dist[s][u]:
                counts[s_idx][u_idx] += 1
            else:
                return

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited, length + 1)
                    visited.remove(nei)

        visited = {s}
        dfs(s, visited, 0)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total

###############################

def clean_csv_nan(filename):
    df = pd.read_csv(filename, dtype=str, encoding="utf-8", keep_default_na=False)

    df = df.replace({
        "nan": "",
        "NaN": "",
        "None": "",
        "NONE": "",
        "Null": "",
        "NULL": "",
        "<NA>": "",
        "pd.NA": ""
    }, regex=False)

    df.to_csv(filename, index=False, encoding="utf-8")

# ============================================================
#  SHARANJE OPTIMUMA v CSV (geodesic)
# ============================================================
def save_to_csv(n, mu, graph, score, direction):
    filename = "2del_geodesic_SA.csv"

    os.makedirs("slike_geodesic_min", exist_ok=True)
    os.makedirs("slike_geodesic_max", exist_ok=True)

    g6 = str(graph.graph6_string())

    # Če CSV ne obstaja, ga ustvari brez NA vrednosti
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n", "µ(G)",
            "min geodesic", "max geodesic",
            "slika_min", "slika_max",
            "graph6_min", "graph6_max"
        ])
        df.to_csv(filename, index=False, encoding="utf-8")

    # Beremo brez NA pretvorbe
    df = pd.read_csv(filename, dtype=str, encoding="utf-8", keep_default_na=False)

    needed = [
        "n", "µ(G)",
        "min geodesic", "max geodesic",
        "slika_min", "slika_max",
        "graph6_min", "graph6_max"
    ]

    for col in needed:
        if col not in df.columns:
            df[col] = ""

    n_str = str(int(n))
    mu_str = str(int(mu))

    mask = (df["n"] == n_str) & (df["µ(G)"] == mu_str)

    # ===================================================
    # 1) NOVA VRSTICA
    # ===================================================
    if not mask.any():
        new_row = {
            "n": n_str,
            "µ(G)": mu_str,

            "min geodesic": str(score) if direction == "min" else "",
            "max geodesic": str(score) if direction == "max" else "",

            "slika_min": f"slike_geodesic_min/graf_geodesic_min_mu{mu}_n{n}.png" if direction == "min" else "",
            "slika_max": f"slike_geodesic_max/graf_geodesic_max_mu{mu}_n{n}.png" if direction == "max" else "",

            "graph6_min": g6 if direction == "min" else "",
            "graph6_max": g6 if direction == "max" else ""
        }

        # Shranimo sliko
        if direction == "min":
            graph.plot().save(new_row["slika_min"])
        else:
            graph.plot().save(new_row["slika_max"])

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(filename, index=False, encoding="utf-8")
        return

    # ===================================================
    # 2) UPDATE OBSTOJEČE VRSTICE
    # ===================================================
    idx = df.index[mask].tolist()[0]

    if direction == "min":
        old = df.at[idx, "min geodesic"]

        if old == "" or score < float(old):
            df.at[idx, "min geodesic"] = str(score)
            df.at[idx, "graph6_min"] = g6

            slika = f"slike_geodesic_min/graf_geodesic_min_mu{mu}_n{n}.png"
            graph.plot().save(slika)
            df.at[idx, "slika_min"] = slika

    else:
        old = df.at[idx, "max geodesic"]

        if old == "" or score > float(old):
            df.at[idx, "max geodesic"] = str(score)
            df.at[idx, "graph6_max"] = g6

            slika = f"slike_geodesic_max/graf_geodesic_max_mu{mu}_n{n}.png"
            graph.plot().save(slika)
            df.at[idx, "slika_max"] = slika

    df.to_csv(filename, index=False, encoding="utf-8")


# ============================================================
#  MUTATE GRAPH
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#  DODAJ 1 VOZLIŠČE
# ============================================================

def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    new_v = max(verts) + 1
    H.add_vertex(new_v)

    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if direction == "min":
        anchor = min(degrees, key=degrees.get)
    else:
        anchor = max(degrees, key=degrees.get)

    H.add_edge(anchor, new_v)
    return H


# ============================================================
#  DODAJ 1 POVEZAVO (najbližji non-edge)
# ============================================================

def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    raise ValueError("Graf je že poln.")


# ============================================================
#  LOAD G(n,µ) IZ CSV
# ============================================================

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("2del_geodesic_SA.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO: prisili int(0) namesto Integer(0)
    row = row.iloc[int(0)]

    g6 = row["graph6_min"] if direction == "min" else row["graph6_max"]
    g6 = str(g6).strip()

    if g6 == "":
        raise ValueError(f"graph6 zapis manjka za n={n_val}, µ={mu_val}")

    return Graph(g6)

# ============================================================
#  SIMULATED ANNEALING – GEODESIC
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=20.0, T_end=0.001,
                        cooling=0.995, max_steps=2000,
                        initial_graph=None):

    G = Graph(initial_graph)

    best_G = G.copy()
    best_score = geodesic_subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = geodesic_subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_G = current_G.copy()
            best_score = current_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#  GONILNA FUNKCIJA
# ============================================================

def compute_all_for_fixed_mu(mu_fix, direction, n_start, n_end, N):

    for n in range(n_start, n_end + 1):

        print(f"Obdelujem: n={n}, µ={mu_fix}")

        m_target = mu_fix + n - 1

        # IZBERI ZAČETNI GRAF
        if n == n_start:
            try:
                Gstart = load_G(n, mu_fix, direction)
            except:
                raise ValueError(f"Manjka začetni graf za n={n}, µ={mu_fix}.")
        else:
            G_prev = load_G(n-1, mu_fix, direction)

            Gstart = add_one_vertex_and_connect(G_prev, direction)

            while Gstart.size() < m_target:
                Gstart = add_one_edge(Gstart)

            while Gstart.size() > m_target:
                edges = list(Gstart.edges())
                e = pyrandom.choice(edges)
                Gstart.delete_edge(e)
                if not Gstart.is_connected():
                    Gstart.add_edge(e)

        best_graph, best_value = simulated_annealing(
            n=n,
            m=m_target,
            direction=direction,
            initial_graph=Gstart,
            max_steps=N
        )

        print(f"=== Rezultat za n={n}, µ={mu_fix} ===")
        print("Best geodesic:", best_value)
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu_fix, best_graph, best_value, direction)


# ============================================================
#  START
# ============================================================

compute_all_for_fixed_mu(
    mu_fix=9,
    direction="max",
    n_start=9,
    n_end=30,
    N=2000
)
